## CSV Besucher-Kunden outer Merge

In [1]:
import pandas as pd

# Load the CSV files with the correct delimiter
besucher_df = pd.read_csv("besucher.csv", delimiter=';')
kunden_df = pd.read_csv("kunden.csv", delimiter=';')

# Replace empty cells in 'Geschlecht' with -1 for both datasets
besucher_df['Geschlecht'].fillna(-1, inplace=True)
kunden_df['Geschlecht'].fillna(-1, inplace=True)

# Remove rows where 'Geschlecht' is -1 from both datasets
besucher_df = besucher_df[besucher_df['Geschlecht'] != -1]
kunden_df = kunden_df[kunden_df['Geschlecht'] != -1]


# Converting "Zeit", "Geschlecht", "Alter", damit Datenformat zwischen csv files das gleiche ist, da wir damit voher Probleme hatten
besucher_df['Zeit'] = besucher_df['Zeit'].str.replace(',', '.').astype(float).astype(int)
kunden_df['Zeit'] = kunden_df ['Zeit'].astype(int)
besucher_df['Geschlecht'] = besucher_df['Geschlecht'].astype(int)
kunden_df['Geschlecht'] = kunden_df ['Geschlecht'].astype(int)
besucher_df['Alter'] = besucher_df['Alter'].astype(int)
kunden_df['Alter'] = kunden_df ['Alter'].astype(int)

# Konvertiert 'Einkommen' zu float, indem Kommas durch Punkte ersetzt werden, dann zu int
besucher_df['Einkommen'] = besucher_df['Einkommen'].astype(str).str.replace(',', '.').astype(float).astype(int)
kunden_df['Einkommen'] = kunden_df['Einkommen'].astype(int)

# Perform an outer merge
merged_df = pd.merge(besucher_df, kunden_df, on=['Alter', 'Einkommen', 'Geschlecht', 'Zeit', 'KundeNr'], how='outer')

# Bereinigen des DAtensatzes, in den Spalten  'Einkommen' und "Alter" 
merged_minus_df = merged_df[(merged_df['Einkommen'] <= 100000000) & (merged_df['Einkommen'] >= 0)& (merged_df["Alter"]<= 90)]

merged_minus_df.to_csv("kunden_besucher_merger.csv", index=False)

## Geo Daten reinigen

In [2]:

# Load the geo.txt file and perform replacements
with open( "geo.txt", "r") as file:
    geo_data = file.read()

# Replacing specified strings in the geo data
geo_data = geo_data.replace('NRW', 'Nordrhein-Westfalen')
geo_data = geo_data.replace('DÃ¼sseldorf', 'Nordrhein-Westfalen')
geo_data = geo_data.replace('BERLIN', 'Berlin')
geo_data = geo_data.replace('Berlin-Charlottenburg', 'Berlin')
geo_data = geo_data.replace('Berlin-Mitte', 'Berlin')


# Saving the modified geo data to a new file
with open('modified_geo.txt', "w") as file: file.write(geo_data)



## CSV Datei mit Geo daten über KundeNr verbunden

In [3]:
##Mergen der CSV-Geo Daten

# Laden der Dateien
file1_path = 'kunden_besucher_merger.csv'  # Pfad zur ersten Datei
file2_path = 'modified_geo.txt'            # Pfad zur zweiten Datei

# Load the modified geo data into a DataFrame
# Assuming that the geo data is in a two-column format with 'KundeNr' and 'Location'
geo_df = pd.read_csv('modified_geo.txt', delimiter=';', names=['KundeNr', 'Location'])

# Lesen der ersten Datei (angenommen, es handelt sich um eine CSV-Datei)
data1 = pd.read_csv(file1_path)

# Lesen der zweiten Datei (angenommen, es handelt sich um eine Textdatei mit Tabulator als Trennzeichen)
data2 = pd.read_csv(file2_path, sep='\t')

# Zusammenführen der Daten
merged_data_df = pd.merge(data1, data2, on='KundeNr', how='inner')

print(merged_data_df.head())





   Alter  Einkommen  Geschlecht  Zeit KundeNr  Preis        Niederlassung
0     30      39919           0    29   K1814    NaN               Bayern
1     41      53430           0    27   K1544    NaN    Baden-Württemberg
2     64      73163           0    32   K1945    NaN    Baden-Württemberg
3     41      36761           0    44   K1825    NaN    Baden-Württemberg
4     35      42684           1    33   K1144    NaN  Nordrhein-Westfalen


In [4]:
#Speichern der Zusammengefügten Datei
merged_data_df.to_csv("df_almost_final.csv", index=False)

## Hinzufügen von Hilfsvariable (Autokauf ja/nein)

In [5]:
# Laden der CSV-Datei mit Komma als Trennzeichen
file_path = "df_almost_final.csv"  
df = pd.read_csv(file_path, sep=',')

# Hinzufügen einer neuen Spalte "Kunde (1=ja)"
# Diese Spalte enthält eine 1, wenn ein Wert in der Spalte "Preis" vorhanden ist,
# und eine 0, wenn kein Wert in "Preis" vorhanden ist
df['Kunde (1=ja)'] = df['Preis'].notna().astype(int)

# Speichern der geänderten Tabelle in einer neuen Datei
df.to_csv('df_final.csv', index=False, sep=",")